In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/events.csv', low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
df_last_15_days = df.loc[df['timestamp'] > pd.to_datetime('2018-05-13')]
df_last_15_days = df_last_15_days.loc[df_last_15_days['event']=='checkout', :]

In [4]:
df_last_15_days.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
33,2018-05-18 00:29:24,checkout,15ea8012,NaN,12802.0,Samsung Galaxy S8 Plus,Excelente,64GB,Prata,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,2018-05-18 00:22:58,checkout,43790d8f,NaN,10378.0,Motorola Moto G4 Plus,Bom,32GB,Bambu,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,2018-05-18 00:38:51,checkout,d614c608,NaN,6343.0,Samsung Galaxy J5,Bom,16GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,2018-05-18 00:57:32,checkout,55d1e0ee,NaN,6663.0,Samsung Galaxy Core Plus Duos TV,Excelente,4GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,2018-05-18 01:12:43,checkout,bb78c182,NaN,7631.0,Motorola Moto G3 4G,Bom,16GB,Preto,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_models = df_last_15_days.groupby(['person', 'model']).agg({'timestamp':'count'})

In [6]:
df_models = df_models.reset_index()
df_models = df_models.pivot_table(index='person', values='timestamp', columns='model')

In [7]:
top_10_models = [
    'Samsung Galaxy J5',
    'iPhone 5s',
    'iPhone 6',
    'Samsung Galaxy S6 Flat',
    'iPhone 6S',
    'iPhone 5c',
    'Motorola Moto G4 Plus',
    'Motorola Moto G3 4G',
    'Samsung Galaxy J7',
    'Samsung Galaxy A5', 
]
df_models = df_models[top_10_models]

In [8]:
df_models = df_models.fillna(0)
df_result = pd.DataFrame()

In [9]:
df_result['total_top_10_phones_checkout'] = df_models.apply('sum', axis=1)
df_result['different_top_10_phones_checkout'] = df_models.astype(bool).sum(axis=1)

In [14]:
user_features = df[['person']].drop_duplicates().copy()
user_features.set_index('person', inplace=True)
user_features = user_features.join(df_result).fillna(0)

In [15]:
user_features.to_csv('../data/top_10_celulares.csv')